# Saga Pattern Performance Test Results Visualization

このノートブックはSagaパターンの性能テスト結果を可視化します。

## 前提条件

以下のCSVファイルは `./saga_pattern/data` 配下の各パターン用サブディレクトリに配置してください：

- `./saga_pattern/data/choreography_pattern/`
  - `e2e_latency.csv`（E2Eレスポンス時間）
  - `convergence_events.csv`（イベント収束）
  - `saga_steps.csv`（サガステップ詳細）

- `./saga_pattern/data/orchestration_pattern/`
  - `e2e_latency.csv`
  - `convergence_events.csv`
  - `saga_steps.csv`
  - `load_phase_results.csv`（ロードフェーズ結果）

- `./saga_pattern/data/single_pessimistic_pattern/`
  - `e2e_latency.csv`
  - `convergence_events.csv`
  - `saga_steps.csv`

各CSVは列スキーマが共通であることを前提に、読み込み後に `pattern` 列で区別します。

In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import numpy as np

# Resolve data directory relative to repo root or notebook location
# Tries common roots to allow running from repo root or notebook folder

def resolve_data_dir():
    candidates = [
        Path('./saga_pattern/data'),                               # from repo root
        Path('../saga_pattern/data'),                              # from saga_pattern notebooks folder
        Path(__file__).parent / 'data' if '__file__' in globals() else Path.cwd() / 'data'
    ]
    for p in candidates:
        if (p / 'choreography_pattern').exists() and (p / 'orchestration_pattern').exists():
            return p.resolve()
    # Fallback to current working directory
    return Path('./saga_pattern/data').resolve()

DATA_DIR = resolve_data_dir()
print(f"データディレクトリ: {DATA_DIR}")
print("ライブラリの読み込み完了")

In [ ]:
# Load CSV data from pattern subdirectories
print("=== CSVファイルの読み込み（3パターン統合）===")

from typing import Dict, List

pattern_dirs = {
    'choreography': DATA_DIR / 'choreography_pattern',
    'orchestration': DATA_DIR / 'orchestration_pattern',
    'single_pessimistic': DATA_DIR / 'single_pessimistic_pattern',
}

# Initialize empty DataFrames
cols_e2e = ['created_at', 'finished_at', 'e2e_ms']
cols_conv = ['aggregate_id', 'processed_at']
cols_saga = ['saga_id', 'step', 'status', 'is_compensation', 'started_at', 'completed_at', 'duration_ms']

df_e2e_list: List[pd.DataFrame] = []
df_conv_list: List[pd.DataFrame] = []
df_saga_list: List[pd.DataFrame] = []

for pattern, pdir in pattern_dirs.items():
    try:
        e2e_path = pdir / 'e2e_latency.csv'
        conv_path = pdir / 'convergence_events.csv'
        saga_path = pdir / 'saga_steps.csv'

        if e2e_path.exists():
            e2e = pd.read_csv(e2e_path, parse_dates=['created_at', 'finished_at'])
            e2e['pattern'] = pattern
            df_e2e_list.append(e2e)
            print(f"✓ E2E ({pattern}): {len(e2e)} rows")
        else:
            print(f"- E2E ファイルなし: {e2e_path}")

        if conv_path.exists():
            conv = pd.read_csv(conv_path, parse_dates=['processed_at'])
            conv['pattern'] = pattern
            df_conv_list.append(conv)
            print(f"✓ Convergence ({pattern}): {len(conv)} rows")
        else:
            print(f"- Convergence ファイルなし: {conv_path}")

        if saga_path.exists():
            saga = pd.read_csv(saga_path, parse_dates=['started_at', 'completed_at'])
            saga['pattern'] = pattern
            df_saga_list.append(saga)
            print(f"✓ Saga ({pattern}): {len(saga)} rows")
        else:
            print(f"- Saga ファイルなし: {saga_path}")

    except Exception as e:
        print(f"読み込みエラー ({pattern}): {e}")

# Concatenate
if df_e2e_list:
    df_e2e = pd.concat(df_e2e_list, ignore_index=True)
else:
    df_e2e = pd.DataFrame(columns=cols_e2e + ['pattern'])

if df_conv_list:
    df_conv = pd.concat(df_conv_list, ignore_index=True)
else:
    df_conv = pd.DataFrame(columns=cols_conv + ['pattern'])

if df_saga_list:
    df_saga = pd.concat(df_saga_list, ignore_index=True)
else:
    df_saga = pd.DataFrame(columns=cols_saga + ['pattern'])

print("\n=== データ概要 ===")
print(f"E2E latency: {len(df_e2e)} rows, columns={list(df_e2e.columns)}")
print(f"Convergence: {len(df_conv)} rows, columns={list(df_conv.columns)}")
print(f"Saga steps: {len(df_saga)} rows, columns={list(df_saga.columns)}")

In [ ]:
# Visualization of performance metrics
def create_performance_graphs(df_e2e, df_conv, df_saga):
    """Create the 3 key performance graphs"""

    if df_e2e.empty or df_conv.empty or df_saga.empty:
        print("データが空のため、グラフを作成できません。CSVファイルを確認してください。")
        return

    plt.style.use('default')
    fig = plt.figure(figsize=(15, 12))

    # 1. E2E Latency Distribution (p50/p95/p99)
    plt.subplot(3, 2, 1)
    if not df_e2e.empty and 'e2e_ms' in df_e2e.columns:
        # Box plot by pattern
        sns.boxplot(data=df_e2e, x='pattern', y='e2e_ms')
        plt.title('E2E Latency Distribution by Pattern')
        plt.ylabel('Latency (ms)')

        # Add percentile lines
        p50 = df_e2e['e2e_ms'].quantile(0.5)
        p95 = df_e2e['e2e_ms'].quantile(0.95)
        p99 = df_e2e['e2e_ms'].quantile(0.99)
        plt.axhline(p50, color='green', linestyle='--', alpha=0.7, label=f'p50={p50:.0f}ms')
        plt.axhline(p95, color='orange', linestyle='--', alpha=0.7, label=f'p95={p95:.0f}ms')
        plt.axhline(p99, color='red', linestyle='--', alpha=0.7, label=f'p99={p99:.0f}ms')
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'No E2E latency data available', ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('E2E Latency Distribution (No Data)')

    # 2. E2E Latency Histogram
    plt.subplot(3, 2, 2)
    if not df_e2e.empty and 'e2e_ms' in df_e2e.columns:
        plt.hist(df_e2e['e2e_ms'], bins=30, alpha=0.7, edgecolor='black')
        plt.title('E2E Latency Histogram')
        plt.xlabel('Latency (ms)')
        plt.ylabel('Frequency')

        # Add percentile markers
        p50 = df_e2e['e2e_ms'].quantile(0.5)
        p95 = df_e2e['e2e_ms'].quantile(0.95)
        plt.axvline(p50, color='green', linestyle='--', label=f'p50={p50:.0f}ms')
        plt.axvline(p95, color='orange', linestyle='--', label=f'p95={p95:.0f}ms')
        plt.legend()
    else:
        plt.text(0.5, 0.5, 'No histogram data available', ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('E2E Latency Histogram (No Data)')

    # 3. Convergence Time Histogram
    plt.subplot(3, 2, 3)
    if not df_conv.empty:
        # Calculate convergence time per aggregate
        conv_summary = df_conv.groupby('aggregate_id').agg({
            'processed_at': ['min', 'max', 'count']
        }).reset_index()
        conv_summary.columns = ['aggregate_id', 'first_event', 'last_event', 'event_count']
        conv_summary['convergence_s'] = (
            conv_summary['last_event'] - conv_summary['first_event']
        ).dt.total_seconds()

        if len(conv_summary) > 0:
            plt.hist(conv_summary['convergence_s'], bins=20, alpha=0.7, edgecolor='black')
            plt.title('Event Convergence Time Histogram')
            plt.xlabel('Convergence Time (seconds)')
            plt.ylabel('Frequency')

            avg_conv = conv_summary['convergence_s'].mean()
            plt.axvline(avg_conv, color='red', linestyle='--', label=f'Avg={avg_conv:.1f}s')
            plt.legend()
        else:
            plt.text(0.5, 0.5, 'No convergence data', ha='center', va='center', transform=plt.gca().transAxes)
            plt.title('Event Convergence (No Data)')
    else:
        plt.text(0.5, 0.5, 'No convergence data available', ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('Event Convergence (No Data)')

    # 4. Compensation Rate and Status
    plt.subplot(3, 2, 4)
    if not df_saga.empty and 'is_compensation' in df_saga.columns:
        # Compensation rate by status
        comp_summary = df_saga.groupby(['status', 'is_compensation']).size().unstack(fill_value=0)
        comp_summary.plot(kind='bar', stacked=True, ax=plt.gca())
        plt.title('Saga Steps: Normal vs Compensation')
        plt.xlabel('Step Status')
        plt.ylabel('Count')
        plt.legend(['Normal', 'Compensation'])
        plt.xticks(rotation=45)
    else:
        plt.text(0.5, 0.5, 'No saga step data available', ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('Saga Steps (No Data)')

    # 5. Saga Timeline (Sample)
    plt.subplot(3, 2, 5)
    if not df_saga.empty and len(df_saga) > 0:
        # Show timeline for first few sagas
        sample_sagas = df_saga['saga_id'].unique()[:5]  # First 5 sagas
        saga_sample = df_saga[df_saga['saga_id'].isin(sample_sagas)].copy()

        if not saga_sample.empty and 'started_at' in saga_sample.columns:
            # Convert to relative time
            base_time = saga_sample['started_at'].min()
            saga_sample['start_rel'] = (saga_sample['started_at'] - base_time).dt.total_seconds()

            # Derive duration seconds whether or not duration_ms exists
            if 'duration_ms' in saga_sample.columns:
                saga_sample['duration_s'] = saga_sample['duration_ms'] / 1000.0
            elif 'completed_at' in saga_sample.columns:
                saga_sample['duration_s'] = (saga_sample['completed_at'] - saga_sample['started_at']).dt.total_seconds().clip(lower=0)
            else:
                plt.text(0.5, 0.5, 'Insufficient duration data', ha='center', va='center', transform=plt.gca().transAxes)
                plt.title('Saga Timeline (Insufficient Data)')
                plt.xlabel('Time (seconds from start)')
                plt.yticks([])
                plt.box(False)
                plt.tight_layout()
                # Early return from this subplot branch
                pass

            # Create Gantt-style chart
            colors = {'COMPLETED': 'green', 'FAILED': 'red', 'COMPENSATED': 'orange', 'PENDING': 'blue'}

            for i, saga_id in enumerate(sample_sagas):
                saga_steps = saga_sample[saga_sample['saga_id'] == saga_id]
                for _, step in saga_steps.iterrows():
                    color = colors.get(step.get('status', ''), 'gray')
                    alpha = 0.8 if bool(step.get('is_compensation', False)) else 0.6
                    left = float(step.get('start_rel', 0))
                    width = float(step.get('duration_s', 0))
                    if width > 0:
                        plt.barh(i, width, left=left, height=0.6, color=color, alpha=alpha)

            plt.yticks(range(len(sample_sagas)), [f"Saga {i+1}" for i in range(len(sample_sagas))])
            plt.xlabel('Time (seconds from start)')
            plt.title('Sample Saga Timeline (Compensation in darker colors)')
        else:
            plt.text(0.5, 0.5, 'Insufficient timeline data', ha='center', va='center', transform=plt.gca().transAxes)
            plt.title('Saga Timeline (Insufficient Data)')
    else:
        plt.text(0.5, 0.5, 'No saga timeline data available', ha='center', va='center', transform=plt.gca().transAxes)
        plt.title('Saga Timeline (No Data)')

    # 6. Summary Statistics
    plt.subplot(3, 2, 6)
    plt.axis('off')

    # Create summary text
    summary_text = "Performance Test Summary\n\n"

    if not df_e2e.empty:
        summary_text += f"E2E Latency (n={len(df_e2e)}):  \n"
        summary_text += f"  p50: {df_e2e['e2e_ms'].quantile(0.5):.1f}ms\n"
        summary_text += f"  p95: {df_e2e['e2e_ms'].quantile(0.95):.1f}ms\n"
        summary_text += f"  p99: {df_e2e['e2e_ms'].quantile(0.99):.1f}ms\n\n"

    if not df_conv.empty:
        conv_summary = df_conv.groupby('aggregate_id').agg({
            'processed_at': ['min', 'max']
        }).reset_index()
        conv_summary.columns = ['aggregate_id', 'first_event', 'last_event']
        conv_summary['convergence_s'] = (conv_summary['last_event'] - conv_summary['first_event']).dt.total_seconds()

        summary_text += f"Convergence (n={len(conv_summary)}):  \n"
        summary_text += f"  Avg: {conv_summary['convergence_s'].mean():.2f}s\n"
        summary_text += f"  p95: {conv_summary['convergence_s'].quantile(0.95):.2f}s\n\n"

    if not df_saga.empty and 'is_compensation' in df_saga.columns:
        comp_rate = df_saga['is_compensation'].mean() * 100
        summary_text += f"Saga Steps (n={len(df_saga)}):  \n"
        summary_text += f"  Compensation rate: {comp_rate:.1f}%\n"
        summary_text += f"  Total compensations: {df_saga['is_compensation'].sum()}\n"

    plt.text(0.1, 0.9, summary_text, fontsize=10, verticalalignment='top',
             transform=plt.gca().transAxes, family='monospace')
    plt.title('Summary Statistics')

    plt.tight_layout()
    plt.show()

# Create performance graphs
print("=== パフォーマンス可視化 ===")
create_performance_graphs(df_e2e, df_conv, df_saga)

print("\n=== 可視化完了 ===")
print("グラフは以下の内容を示しています:")
print("1. E2E latency distribution (低レイテンシー)")
print("2. Event convergence times (結果整合性)")
print("3. Compensation timeline (障害処理)")

## グラフの説明

### 1. E2E Latency Distribution
- **目的**: Sagaパターンの低レイテンシー特性を示す
- **左上**: パターン別（Choreography vs Orchestration）のレイテンシー分布
- **右上**: 全体のレイテンシーヒストグラム（p50/p95線付き）

### 2. Event Convergence Time
- **目的**: 結果整合性（Eventual Consistency）の収束時間を示す
- **左中**: イベント発行から最終状態反映までの時間分布
- **意味**: 数秒以内に収束すれば良好

### 3. Saga Timeline & Compensation
- **目的**: 補償処理の動作を可視化
- **右中**: サガステップ（正常 vs 補償）の件数
- **左下**: サンプルサガのタイムライン（補償ステップは濃い色）
- **右下**: 統計サマリー

## 重要指標

- **低レイテンシー**: p95 < 1000ms が目安
- **収束時間**: 平均 < 10秒 が目安  
- **補償率**: 5-15% が一般的（失敗注入率に依存）

In [ ]:
# Load phase-specific analysis (if available)
def load_phase_analysis():
    """Load and analyze phase-specific load test results"""
    print("=== Load Phase Analysis ===")

    try:
        # phase results are only produced for orchestration in current setup
        phase_path = DATA_DIR / 'orchestration_pattern' / 'load_phase_results.csv'
        df_phases = pd.read_csv(phase_path, parse_dates=['timestamp'])
        print(f"✓ Load phase data: {len(df_phases)} rows from {phase_path}")

        # Phase comparison visualization
        plt.figure(figsize=(15, 10))

        # 1. Response time by phase (box plot)
        plt.subplot(2, 3, 1)
        sns.boxplot(data=df_phases, x='load_phase', y='response_time', order=['light', 'medium', 'heavy'])
        plt.title('Response Time by Load Phase')
        plt.ylabel('Response Time (s)')

        # 2. Success rate by phase
        plt.subplot(2, 3, 2)
        success_by_phase = df_phases.groupby('load_phase')['status_code'].apply(
            lambda x: (x.isin([200, 201])).mean() * 100
        ).reindex(['light', 'medium', 'heavy'])
        success_by_phase.plot(kind='bar')
        plt.title('Success Rate by Load Phase')
        plt.ylabel('Success Rate (%)')
        plt.xticks(rotation=45)

        # 3. Request count by phase
        plt.subplot(2, 3, 3)
        count_by_phase = df_phases['load_phase'].value_counts().reindex(['light', 'medium', 'heavy'])
        count_by_phase.plot(kind='bar', color=['lightblue', 'orange', 'red'])
        plt.title('Request Count by Load Phase')
        plt.ylabel('Request Count')
        plt.xticks(rotation=45)

        # 4. Response time over time (timeline)
        plt.subplot(2, 3, 4)
        df_phases['timestamp_dt'] = pd.to_datetime(df_phases['timestamp'])
        for phase in ['light', 'medium', 'heavy']:
            phase_data = df_phases[df_phases['load_phase'] == phase]
            if not phase_data.empty:
                plt.scatter(phase_data['timestamp_dt'], phase_data['response_time'],
                           label=phase, alpha=0.6, s=20)
        plt.title('Response Time Timeline by Phase')
        plt.ylabel('Response Time (s)')
        plt.xlabel('Time')
        plt.legend()
        plt.xticks(rotation=45)

        # 5. P95/P99 comparison across phases
        plt.subplot(2, 3, 5)
        percentiles = df_phases.groupby('load_phase')['response_time'].agg([
            lambda x: x.quantile(0.50),
            lambda x: x.quantile(0.95),
            lambda x: x.quantile(0.99)
        ]).reindex(['light', 'medium', 'heavy'])
        percentiles.columns = ['P50', 'P95', 'P99']
        percentiles.plot(kind='bar')
        plt.title('Response Time Percentiles by Phase')
        plt.ylabel('Response Time (s)')
        plt.xticks(rotation=45)
        plt.legend()

        # 6. Pattern distribution by phase
        plt.subplot(2, 3, 6)
        if 'pattern' in df_phases.columns:
            pattern_phase = pd.crosstab(df_phases['load_phase'], df_phases['pattern'], normalize='index') * 100
            pattern_phase.reindex(['light', 'medium', 'heavy']).plot(kind='bar', stacked=True)
            plt.title('Pattern Distribution by Phase (%)')
            plt.ylabel('Percentage')
            plt.xticks(rotation=45)
            plt.legend()
        else:
            plt.axis('off')
            plt.text(0.1, 0.8, 'No pattern breakdown in phase dataset', transform=plt.gca().transAxes)

        plt.tight_layout()
        plt.show()

        # Summary statistics
        print("\n=== Phase Summary Statistics ===")
        phase_stats = df_phases.groupby('load_phase').agg({
            'response_time': ['count', 'mean', 'std', lambda x: x.quantile(0.95), lambda x: x.quantile(0.99)],
            'status_code': lambda x: (x.isin([200, 201])).mean() * 100
        }).round(3)
        phase_stats.columns = ['Count', 'Avg_RT(s)', 'Std_RT', 'P95_RT(s)', 'P99_RT(s)', 'Success_Rate(%)']
        print(phase_stats.reindex(['light', 'medium', 'heavy']))

        return df_phases

    except FileNotFoundError:
        print("load_phase_results.csv not found under orchestration_pattern. Multi-phase test may not have been run.")
        return None
    except Exception as e:
        print(f"Error loading phase data: {e}")
        return None

# Run phase analysis
df_phase_results = load_phase_analysis()